In [1]:
import os
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.stats import norm
from itertools import cycle
from random import shuffle
from sklearn.model_selection import train_test_split



import torch
from torch import nn
import torch.optim as opt
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from livelossplot import PlotLosses
from mpl_toolkits.axes_grid1 import ImageGrid
from collections import OrderedDict

from cycle_consistent_vae_in import Encoder, Decoder
from latent_classifier import Classifier


np.random.bit_generator = np.random._bit_generator

In [2]:
cuda = 1
device = torch.device("cuda:{}".format(cuda) if torch.cuda.is_available() else "cpu")

MODEL_PATH = "./models_final/cycle_vae_512_64_in_13052020-175056_499.pth"
checkpoint = torch.load(MODEL_PATH)

Z_DIM = 512 #Style Dimension (Unspecified)
S_DIM = 64 # Class Dimension (Specified)

encoder = Encoder(style_dim=Z_DIM, class_dim=S_DIM)
encoder.load_state_dict(checkpoint['encoder'])

encoder.to(device)
# encoder.eval()

Encoder(
  (conv_model): Sequential(
    (conv_1): Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (bn_1): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (relu_1): ReLU(inplace)
    (conv_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (bn_2): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (relu_2): ReLU(inplace)
    (conv_3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (bn_3): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (relu_3): ReLU(inplace)
    (conv_4): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (bn_4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (relu_4): ReLU(inplace)
  )
  (style_mu): Linear(in_features=512, out_features=512, bias=True)
  (style_logvar): Linear(in_features=512, out_features=512, bias=True)
  (c

In [3]:
def reparameterize(training, mu, logvar):
    if training:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu
    else:
        return mu

# Dataset processing

In [4]:
class Latent(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.encoder = encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img = self.data[index][:,:,:3]
        
        return (img, self.labels[index])

In [7]:
# PATH = './processed/'
# files = os.listdir(PATH)
# data = []
# labels = []

# counter=0
# for folder in files:
#     print(counter)
#     sprites = os.listdir(PATH+folder)
#     for sprite in sprites:
#         data.append(plt.imread(PATH+folder+"/"+sprite))
#         labels.append(int(folder))
#     counter+=1

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(data, labels, \
#                                                     test_size=0.33, random_state=42, \
#                                                     shuffle=True, stratify=labels )

In [9]:
# train_data = Latent(X_train, y_train)
# test_data = Latent(X_test, y_test)

In [10]:
# with open('train_latent.pkl','wb') as f:
#     pickle.dump(train_data, f)
# with open('test_latent.pkl','wb') as f:
#     pickle.dump(test_data, f)

In [5]:
t1 = pickle.load(open('train_latent.pkl','rb'))
t2 = pickle.load(open('test_latent.pkl','rb'))

In [6]:
BATCH_SIZE = 16
train_loader = DataLoader(t1,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(t2,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

In [7]:
NUM_CLASSES=672

# Latent Classifier - Unpecified Features

In [ ]:
TIME_STAMP = time.strftime("%d%m%Y-%H%M%S")

NUM_EPOCHS = 200

LEARNING_RATE = 1e-3

MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
CUDA = True

In [ ]:
model = Classifier(z_dim=Z_DIM, num_classes=NUM_CLASSES)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                                 lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [ ]:
is_better = True
prev_acc = 0
name = "unspecified_latent_classifier_512_65_in_499"

liveloss = PlotLosses(fig_path='./figures/'+name+".png")

In [ ]:
dataloaders = {'train':train_loader, 'validation':test_loader}

In [ ]:
for epoch in range(NUM_EPOCHS):
    logs = {}
    t_start = time.time()
    
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
            
        else:
            model.eval()
        model.to(device)
        
        print("Started Phase")

        running_loss = 0.0
                
        predicted_phase = torch.zeros(len(dataloaders[phase].dataset), NUM_CLASSES)
        target_phase = torch.zeros(len(dataloaders[phase].dataset))
        
        if phase == 'validation':
            
            with torch.no_grad():
                
                for (i,batch) in enumerate(dataloaders[phase]):
                    input_tensor = batch[0]
                    input_tensor = input_tensor.to(device)
                    input_tensor = torch.transpose(input_tensor, 2,3)
                    input_tensor = torch.transpose(input_tensor, 1,2)

                    z_mu, z_logvar, _ = encoder(input_tensor)
                    latent_vector = reparameterize(training=True, mu=z_mu, logvar=z_logvar)

                    bs = input_tensor.shape[0]
                    target_tensor = batch[1].to(device).reshape(bs)

                    softmaxed_tensor = model(latent_vector)

                    loss = criterion(softmaxed_tensor, target_tensor.long())

                    predicted_phase[i*bs:(i+1)*bs] = softmaxed_tensor.cpu()
                    target_phase[i*bs:(i+1)*bs] = target_tensor.cpu()

                    input_tensor = input_tensor.cpu()
                    target_tensor = target_tensor.cpu()

                    running_loss += loss.detach() * bs
                
     
        else:
            
            for (i,batch) in enumerate(dataloaders[phase]):
                with torch.no_grad():
                    input_tensor = batch[0]
                    input_tensor = input_tensor.to(device)
                    input_tensor = torch.transpose(input_tensor, 2,3)
                    input_tensor = torch.transpose(input_tensor, 1,2)

                    z_mu, z_logvar, _ = encoder(input_tensor)
                    latent_vector = reparameterize(training=True, mu=z_mu, logvar=z_logvar)
                
                bs = input_tensor.shape[0]
                target_tensor = batch[1].to(device).reshape(bs)
                
                softmaxed_tensor = model(latent_vector)

                loss = criterion(softmaxed_tensor, target_tensor.long())

                predicted_phase[i*bs:(i+1)*bs] = softmaxed_tensor.cpu()
                target_phase[i*bs:(i+1)*bs] = target_tensor.cpu()
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


                input_tensor = input_tensor.cpu()
                target_tensor = target_tensor.cpu()

                running_loss += loss.detach() * bs
    

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_predicted = torch.argmax(predicted_phase, dim=1)
        epoch_f1 = f1_score(target_phase, epoch_predicted, average = 'macro')
        epoch_accuracy = accuracy_score(target_phase, epoch_predicted)

        
        model.to('cpu')

        prefix = ''
        if phase == 'validation':
            prefix = 'val_'

        logs[prefix + 'log loss'] = epoch_loss.item()
        logs[prefix + 'f1'] = epoch_f1.item()
        logs[prefix + 'accuracy'] = epoch_accuracy.item()
        
        print('Phase time - ',time.time() - t_start)

    delta = time.time() - t_start
    is_better = logs['val_accuracy'] > prev_acc
    if is_better:
        prev_acc = logs['val_accuracy']
        torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(), 'loss': logs['log loss']}, "./models/"+name+"_"+TIME_STAMP+"_"+str(logs['val_accuracy'])+".pth")


    liveloss.update(logs)
    liveloss.draw()

# Inference


In [8]:
CUDA = True

In [9]:
UNSPECIFIED_MODEL_PATH = './models_final/unspecified_latent_classifier_512_65_in_499_14052020-234301_0.0025079799361605107.pth'
checkpoint = torch.load(UNSPECIFIED_MODEL_PATH)

criterion = torch.nn.CrossEntropyLoss()

model = Classifier(z_dim=Z_DIM, num_classes=NUM_CLASSES)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Classifier(
  (fc_model): Sequential(
    (fc_1): Linear(in_features=512, out_features=2048, bias=True)
    (leakyrelu_1): LeakyReLU(negative_slope=0.2, inplace)
    (fc_2): Linear(in_features=2048, out_features=1024, bias=True)
    (leakyrelu_2): LeakyReLU(negative_slope=0.2, inplace)
    (fc_3): Linear(in_features=1024, out_features=672, bias=True)
  )
)

In [ ]:
logs = {}
t_start = time.time()
predicted_phase = torch.zeros(len(test_loader.dataset), NUM_CLASSES)
target_phase = torch.zeros(len(test_loader.dataset))
running_loss = 0.0
with torch.no_grad():
    for (i,batch) in enumerate(test_loader):
        with torch.no_grad():
            input_tensor = batch[0]
#             input_tensor = input_tensor.to(device)
            input_tensor = torch.transpose(input_tensor, 2,3)
            input_tensor = torch.transpose(input_tensor, 1,2)

            z_mu, z_logvar, _ = encoder(input_tensor.to(device))
            latent_vector = reparameterize(training=True, mu=z_mu, logvar=z_logvar)

        bs = input_tensor.shape[0]
        target_tensor = batch[1].to(device).reshape(bs)

        softmaxed_tensor = model(latent_vector.cpu())

        loss = criterion(softmaxed_tensor, target_tensor.cpu())

        predicted_phase[i*bs:(i+1)*bs] = softmaxed_tensor.cpu()
        target_phase[i*bs:(i+1)*bs] = target_tensor.cpu()

        input_tensor = input_tensor.cpu()
        target_tensor = target_tensor.cpu()

        running_loss += loss.detach() * bs
epoch_loss = running_loss / len(test_loader.dataset)
epoch_predicted = torch.argmax(predicted_phase, dim=1)
epoch_f1 = f1_score(target_phase, epoch_predicted, average = 'macro')
epoch_accuracy = accuracy_score(target_phase, epoch_predicted)

In [ ]:
epoch_accuracy